# Not enough chunks

In [ ]:
import pandas as pd
import nannyml as nml
reference, analysis, _ = nml.datasets.load_synthetic_binary_classification_dataset()
# metadata = nml.extract_metadata(reference, model_type='classification_binary', exclude_columns=['identifier'])
# metadata.target_column_name = 'work_home_actual'
cbpe = nml.CBPE(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='work_home_actual',
    timestamp_column_name='timestamp',
    chunk_number=8,
    metrics=['roc_auc'],
    problem_type='classification_binary',
).fit(reference_data=reference)
# Estimate on concatenated reference and analysis
est_perf = cbpe.estimate(pd.concat([reference, analysis], ignore_index=True))
# est_perf = cbpe.estimate(analysis)
est_perf.data#.iloc[3:5,:7]

,key,start_index,end_index,start_date,end_date,realized_roc_auc,estimated_roc_auc,upper_confidence_roc_auc,lower_confidence_roc_auc,sampling_error_roc_auc,upper_threshold_roc_auc,lower_threshold_roc_auc,alert_roc_auc,period,estimated
0,[0:12499],0,12499,2014-05-09 22:27:20,2015-03-10 03:50:28,0.972972,0.968923,0.972359,0.965487,0.001145,0.977452,0.964553,False,analysis,True
1,[12500:24999],12500,24999,2015-03-10 04:37:25,2016-01-08 16:02:05,0.970533,0.968914,0.972350,0.965479,0.001145,0.977452,0.964553,False,analysis,True
2,[25000:37499],25000,37499,2016-01-08 17:22:00,2016-11-03 19:17:29,0.970198,0.968930,0.972366,0.965495,0.001145,0.977452,0.964553,False,analysis,True
3,[37500:49999],37500,49999,2016-11-03 19:18:08,2017-08-31 03:10:29,0.970788,0.969044,0.972480,0.965608,0.001145,0.977452,0.964553,False,analysis,True
4,[50000:62499],50000,62499,2017-08-31 04:20:00,2018-07-02 16:13:22,NaN,0.969050,0.972485,0.965614,0.001145,0.977452,0.964553,False,analysis,True
5,[62500:74999],62500,74999,2018-07-02 16:15:16,2019-04-30 11:01:30,NaN,0.969032,0.972467,0.965596,0.001145,0.977452,0.964553,False,analysis,True
6,[75000:87499],75000,87499,2019-04-30 11:02:00,2020-03-01 22:53:25,NaN,0.960740,0.964176,0.957305,0.001145,0.977452,0.964553,True,analysis,True
7,[87500:99999],87500,99999,2020-03-01 23:25:08,2021-01-01 04:29:32,NaN,0.961128,0.964564,0.957693,0.001145,0.977452,0.964553,True,analysis,True


In [ ]:
import pandas as pd
import nannyml as nml
reference, analysis, _ = nml.datasets.load_synthetic_binary_classification_dataset()

cbpe = nml.CBPE(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='work_home_actual',
    timestamp_column_name='timestamp',
    chunk_period="Q",
    metrics=['roc_auc'],
    problem_type='classification_binary',
).fit(reference_data=reference)
est_perf = cbpe.estimate(analysis)

In [ ]:
cbpe = nml.CBPE(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='work_home_actual',
    timestamp_column_name='timestamp',
    chunk_number=5,
    metrics=['roc_auc'],
    problem_type='classification_binary',
).fit(reference_data=reference)
est_perf = cbpe.estimate(analysis)

/home/niels/Code/nml/nannyml/nannyml/nannyml/chunk.py:157: UserWarning: The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.
  warnings.warn(
/home/niels/Code/nml/nannyml/nannyml/nannyml/chunk.py:157: UserWarning: The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.
  warnings.warn(


/home/niels/Code/nml/nannyml/nannyml/nannyml/chunk.py:157: UserWarning: The resulting number of chunks is too low. Please consider splitting your data in a different way or continue at your own risk.
  warnings.warn(


# Not enough observations in chunk

# Sampling error

In [ ]:
save_figs = False

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

sample_size = 100
dataset_size = 10_000
# random model
np.random.seed(23)
y_true = np.random.binomial(1, 0.5, dataset_size)
y_pred = np.random.binomial(1, 0.5, dataset_size)
accuracy_scores = []

for experiment in range(1000):
    subset_indexes = np.random.choice(dataset_size, sample_size, replace=False) # get random indexes
    y_true_subset = y_true[subset_indexes]
    y_pred_subset = y_pred[subset_indexes]
    accuracy_scores.append(accuracy_score(y_true_subset, y_pred_subset))

plt.hist(accuracy_scores, bins=20, density=True)
plt.title("Accuracy of random classifier\n for randomly selected samples of 100 observations.");
if save_figs:
    plt.savefig("../_static/deep_dive_data_chunks_stability_of_accuracy.svg", format="svg")

In [ ]:
obs_level_accuracy = y_true == y_pred
np.mean(obs_level_accuracy), accuracy_score(y_true, y_pred)

(0.5045, 0.5045)

In [ ]:
SEM_std = np.std(obs_level_accuracy)/np.sqrt(sample_size)
np.round(SEM_std, 3), np.round(np.std(accuracy_scores), 3)

(0.05, 0.049)